In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import *
import warnings
warnings.filterwarnings("ignore")
from colorama import Fore, Style
from sklearn.tree import DecisionTreeRegressor  
from sklearn.ensemble import RandomForestRegressor 
from sklearn.tree import plot_tree 


In [ ]:
train_df=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_df=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
sub=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
train_df.head(5)

In [ ]:
train_df.drop('Id',axis=1,inplace=True)
test_df.drop('Id',axis=1,inplace=True)

In [ ]:
pd.set_option('max_colwidth', None)

In [ ]:
print(Fore.BLUE,train_df.columns.to_list())

In [ ]:
with open('../input/house-prices-advanced-regression-techniques/data_description.txt', encoding='utf8') as f:
    for line in f:
        print(Fore.GREEN + line.strip())
print(Style.RESET_ALL)


In [ ]:
train_df.info()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
train_df.describe()

In [ ]:
test_df.describe()

In [ ]:
sns.heatmap(train_df.isnull(), cbar=False);

In [ ]:
import missingno as msno
msno.matrix(train_df, sort="descending", figsize=(25, 10), width_ratios=(15, 1), color=(0.4, 0.7, 0.15), fontsize=12, labels=True, label_rotation=90, sparkline=True);

In [ ]:
columns=train_df.columns.tolist()
for column in columns:
    sns.histplot(train_df[column]);

In [ ]:
train_df.hist(bins=60,figsize=(20,20));

In [ ]:
print(Fore.LIGHTGREEN_EX,train_df.isnull().sum().to_dict())

In [ ]:
print(Fore.LIGHTGREEN_EX,test_df.isnull().sum().to_dict())

In [ ]:
x=train_df.drop("SalePrice",axis=1)
y=train_df["SalePrice"]

In [ ]:
numerical_columns = [c for c in x.columns if x[c].dtype != 'O']
x_n = x[numerical_columns].copy()
test_n = test_df[numerical_columns].copy()

categorical_columns = [c for c in x.columns if x[c].dtype == 'O']
x_c = x[categorical_columns].copy()
test_c = test_df[categorical_columns].copy()

In [ ]:
x_n

In [ ]:
column_means = x_n.mean()

In [ ]:
x_n=x_n.fillna(column_means)

In [ ]:
corr = x_n.iloc[:,2:].corr()
colormap = sns.diverging_palette(220, 10, as_cmap = True)
plt.figure(figsize=(14,14))
sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 8},
            cmap = colormap, linewidths=0.1, linecolor='white')
plt.title('Correlation of Features', y=1.05, size=15)

In [ ]:
sns.clustermap(x_n.corr(), cmap="coolwarm")

In [ ]:
x_n.corr().style.background_gradient(cmap='coolwarm')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(x_n, y)
importance = rf.feature_importances_
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

plt.bar([a for a in range(len(importance))], importance)
plt.show()

In [ ]:
import xgboost as xgb
from xgboost import plot_importance
model = xgb.XGBRegressor()
model.fit(x_n, y)
plot_importance(model)
plt.rcParams['font.size'] = 5  
plt.show()

In [ ]:
print(x_n.columns.to_list())

In [ ]:
x=x_n[["LotArea","LotFrontage","MSSubClass","BsmtUnfSF","GarageArea","GrLivArea","BsmtFinSF1","TotalBsmtSF","YearBuilt","1stFlrSF","MoSold"]]

In [ ]:
x

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.9, random_state=1)

In [ ]:
model = xgb.XGBRegressor()
model.fit(x_test, y_test)

In [ ]:
test=test_df[["LotArea","LotFrontage","MSSubClass","BsmtUnfSF","GarageArea","GrLivArea","BsmtFinSF1","TotalBsmtSF","YearBuilt","1stFlrSF","MoSold"]]

In [ ]:
col_mean=test.mean()
test=test.fillna(col_mean)

In [ ]:
pred=model.predict(test)

In [ ]:
sub['SalePrice']=pred
sub.to_csv('submission.csv', index=False)